In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import re
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.datasets import imdb

from keras.utils.np_utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
data=pd.read_csv("D:/deva/spam.csv",encoding="latin_1")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [106]:
data=data[["v1","v2"]]
data.head(20)

KeyError: "None of [Index(['v1', 'v2'], dtype='object')] are in the [columns]"

In [23]:
data=data.rename(columns={"v1":"Label","v2":"Mail"})

In [109]:
data.tail(50)

,Label,Mail
5522,ham,thats cool i want to please you
5523,ham,going to join tomorrow
5524,spam,you are awarded a sipix digital camera call 09...
5525,ham,i want to tell you how bad i feel that basical...
5526,spam,private your 2003 account statement for shows ...
5527,ham,total disappointment when i texted you was the...
5528,ham,its just the effect of irritation just ignore it
5529,ham,what about this one then
5530,ham,i think that tantrums finished so yeah ill be ...
5531,ham,compliments to you was away from the system ho...


In [111]:
data.iloc[5524:5525]

,Label,Mail
5524,spam,you are awarded a sipix digital camera call 09...


In [102]:
data["Label"].iloc[5566:5567]

5566    spam
Name: Label, dtype: object

In [25]:
data['Mail'] = data['Mail'].apply(lambda x: x.lower())
data['Mail'] = data['Mail'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [26]:
data.head()

,Label,Mail
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [28]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Mail'].values)
x = tokenizer.texts_to_sequences(data['Mail'].values)
x = pad_sequences(x)

In [30]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
y=data[["Label"]].apply(lb.fit_transform)


In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)

In [37]:
model = Sequential()
model.add(Embedding(max_fatures, 244 ,input_length = x.shape[1]))
model.add(LSTM(244))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [38]:

batch_size = 16
model.fit(x_train, y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
244/244 - 51s - loss: 0.1262 - accuracy: 0.9592
Epoch 2/5
244/244 - 62s - loss: 0.0346 - accuracy: 0.9895
Epoch 3/5
244/244 - 64s - loss: 0.0158 - accuracy: 0.9956
Epoch 4/5
244/244 - 67s - loss: 0.0100 - accuracy: 0.9977
Epoch 5/5
244/244 - 68s - loss: 0.0214 - accuracy: 0.9938


In [39]:
model.evaluate(x_test,y_test,verbose=2,batch_size=batch_size)

105/105 - 5s - loss: 0.1028 - accuracy: 0.9743


[0.10284740477800369, 0.9742823243141174]

In [125]:
a=model.predict(x[5540:5541])
a=np.round(a)
if a==0:
    print("Good Mail")
else:
    print("Spam Mail")

Spam Mail
